# Naive Bayes
### For data with nominal attribute
Given the training data in the table below (Buy Computer data), predict the class of   the   following   new   example   using   Naïve   Bayes   classification: age<=30, income=medium, student=yes, credit-rating=fair

In [3]:
%cd "/home/mona/3074 ML Lab/Datasets"

/home/mona/3074 ML Lab/Datasets


In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math

df = pd.read_csv('buy_computer.csv')

le = LabelEncoder()

dataset = df.apply(le.fit_transform)

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

dataset

,age,income,student,credit_rating,buys_computer
0,1,0,0,1,0
1,1,0,0,0,0
2,0,0,0,1,1
3,2,2,0,1,1
4,2,1,1,1,1
5,2,1,1,0,0
6,0,1,1,0,1
7,1,2,0,1,0
8,1,1,1,1,1
9,2,2,1,1,1


In [6]:
def groupUnderClass(mydata):
    dict = {}
    for i in range(len(mydata)):
        if (mydata.iloc[i, -1] not in dict):
            dict[mydata.iloc[i, -1]] = []
        dict[mydata.iloc[i, -1]].append(mydata.iloc[i, :])
    return dict

def mean(numbers):
    return sum(numbers) / float(len(numbers))

def std_dev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)

def MeanAndStdDev(mydata):
    info = [(mean(attribute), std_dev(attribute)) for attribute in zip(*mydata)]
    del info[-1]
    return info

def MeanAndStdDevForClass(mydata):
    info = {}
    dict = groupUnderClass(mydata)
    for classValue, instances in dict.items():
        info[classValue] = MeanAndStdDev(instances)
    return info

def calculateGaussianProbability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * expo

def calculateClassProbabilities(info, test):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = test[i]
            probabilities[classValue] *= calculateGaussianProbability(x, mean, std_dev)
    return probabilities

def predict(info, test):
    probabilities = calculateClassProbabilities(info, test)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(info, test):
    predictions = []
    for i in range(len(test)):
        result = predict(info, test.iloc[i, :])
        predictions.append(result)
    return predictions

def accuracy_rate(test, predictions):
    correct = 0
    for i in range(len(test)):
        if test.iloc[i] == predictions[i]:
            correct += 1
    return (correct / float(len(test))) * 100.0

In [7]:
info = MeanAndStdDevForClass(dataset)
predictions = getPredictions(info, X)

accuracy = accuracy_rate(y, predictions)
print("Accuracy of Naive Bayes Model is: ", accuracy)

Accuracy of Naive Bayes Model is:  85.71428571428571


In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_true = y
y_pred = predictions
print('Confusion Matrix: \n', confusion_matrix(y_true, y_pred))

tp, fn, fp, tn = confusion_matrix(y_true,y_pred,labels=[0,1]).reshape(-1)
print('\nOutcome values : \n', tp, fn, fp, tn)

matrix = classification_report(y_true,y_pred,labels=[0,1])
print('\nClassification report : \n',matrix)

Confusion Matrix: 
 [[4 1]
 [1 8]]

Outcome values : 
 4 1 1 8

Classification report : 
               precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           1       0.89      0.89      0.89         9

    accuracy                           0.86        14
   macro avg       0.84      0.84      0.84        14
weighted avg       0.86      0.86      0.86        14



In [11]:
print('For the Data Instance \nX = (age <=30, Income = medium, Student = yes, Credit_rating = fair)\n')
X_test = pd.DataFrame([[1, 2, 1, 1]])

predictions = getPredictions(info, X_test)

if predictions[0] == 1:
    print('Prediction is: Yes the student will buy computer')
else:
    print('Prediction is: No the student will not buy computer')

For the Data Instance 
X = (age <=30, Income = medium, Student = yes, Credit_rating = fair)

Prediction is: Yes the student will buy computer


## Spot
Classify audio files using Naive Bayes

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math

df = pd.read_csv('voice.csv')

le = LabelEncoder()

dataset = df.apply(le.fit_transform)

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

dataset.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,3,2243,6,35,3,1305,3063,3059,1379,2151,...,3,66,44,117,0,1,0,0,0,1
1,5,2392,9,45,4,1292,3090,3100,1347,2272,...,5,485,52,95,5,1,3,7,63,1
2,8,2964,8,24,11,2616,3155,3155,516,2072,...,8,267,41,113,2,1,1,1,36,1
3,464,2580,562,485,736,2165,258,288,3018,2990,...,464,95,166,95,281,1,67,79,2481,1
4,248,2826,214,329,647,2714,139,327,3136,3135,...,248,447,119,109,1369,1,624,665,2280,1


In [32]:
def groupUnderClass(mydata):
    dict = {}
    for i in range(len(mydata)):
        if (mydata.iloc[i, -1] not in dict):
            dict[mydata.iloc[i, -1]] = []
        dict[mydata.iloc[i, -1]].append(mydata.iloc[i, :])
    return dict

def mean(numbers):
    return sum(numbers) / float(len(numbers))

def std_dev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)

def MeanAndStdDev(mydata):
    info = [(mean(attribute), std_dev(attribute)) for attribute in zip(*mydata)]
    del info[-1]
    return info

def MeanAndStdDevForClass(mydata):
    info = {}
    dict = groupUnderClass(mydata)
    for classValue, instances in dict.items():
        info[classValue] = MeanAndStdDev(instances)
    return info

def calculateGaussianProbability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * expo

def calculateClassProbabilities(info, test):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = test[i]
            probabilities[classValue] *= calculateGaussianProbability(x, mean, std_dev)
    return probabilities

def predict(info, test):
    probabilities = calculateClassProbabilities(info, test)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(info, test):
    predictions = []
    for i in range(len(test)):
        result = predict(info, test.iloc[i, :])
        predictions.append(result)
    return predictions

def accuracy_rate(test, predictions):
    correct = 0
    for i in range(len(test)):
        if test.iloc[i] == predictions[i]:
            correct += 1
    return (correct / float(len(test))) * 100.0

In [33]:
info = MeanAndStdDevForClass(dataset)
predictions = getPredictions(info, X)

accuracy = accuracy_rate(y, predictions)
print("Accuracy of Naive Bayes Model is: ", accuracy)

Accuracy of Naive Bayes Model is:  87.62626262626263


In [34]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_true = y
y_pred = predictions
print('Confusion Matrix: \n', confusion_matrix(y_true, y_pred))

tp, fn, fp, tn = confusion_matrix(y_true,y_pred,labels=[0,1]).reshape(-1)
print('\nOutcome values : \n', tp, fn, fp, tn)

matrix = classification_report(y_true,y_pred,labels=[0,1])
print('\nClassification report : \n',matrix)

Confusion Matrix: 
 [[1320  264]
 [ 128 1456]]

Outcome values : 
 1320 264 128 1456

Classification report : 
               precision    recall  f1-score   support

           0       0.91      0.83      0.87      1584
           1       0.85      0.92      0.88      1584

    accuracy                           0.88      3168
   macro avg       0.88      0.88      0.88      3168
weighted avg       0.88      0.88      0.88      3168



In [39]:
print('For the Data Instance: \n')
X_test. = pd.DataFrame([[474, 2980, 522, 385, 716, 1165, 251, 188, 3008, 1990, 0, 464, 55, 166, 65, 281, 1, 57, 59, 3481, 1]])

predictions = getPredictions(info, X_test)

if predictions[0] == 1:
    print('Prediction is: Voice belongs to a male')
else:
    print('Prediction is: Voice belongs to a female')

For the Data Instance: 

Prediction is: Voice belongs to a male
